In [1]:
%matplotlib inline
import requests
import json
import time
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

In [2]:
# Hàm thu thập dữ liệu từ một khoảng thời gian cụ thể
def collect_data(api_url):
    weather_data = []
    success = False
    while not success:
        print(f"GET: {api_url}")
        r = requests.get(api_url)
        if r.ok:
            # lấy kết quả json
            pydata = json.loads(r.text)
            weather_data.extend(pydata["days"])  # Lưu dữ liệu từ 'days'
            print("Successful! Remaining cost:", pydata.get('remainingCost', 'Unknown'))
            print()
            success = True
        else:
            print("Fail! Try again. Status code:", r.status_code)
            time.sleep(5)
        
    return weather_data


In [3]:
# Hàm tạo URL API từ các tham số đầu vào
def create_api_url(location, start_date, end_date, api_key, unit_group='metric', include='days'):
    return f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}?unitGroup={unit_group}&include={include}&key={api_key}&contentType=json'

# Hàm thu thập dữ liệu trong khoảng thời gian từ nhiều năm
def collect_all_data(start_date, end_date, location, api_key):
    # Tạo URL API
    api_url = create_api_url(location, start_date, end_date, api_key)
    # Thu thập dữ liệu thời tiết
    weather_data = collect_data(api_url)
    
    return weather_data

In [4]:
# API key (có thể thay đổi)
api_key1 = 'TE7HPVTQ4Q4B7LWSLDSU38BRF'
api_key2 = "VNTCFNZ5VLGZC3FW4F85U8425"
api_key3 = "VWAK37JBEPXKWXTP7U8MQG9FR"

# Thông số đầu vào (có thể thay đổi)
location = 'hanoi'
start_date = '2022-09-01'
end_date = '2024-09-01'

In [5]:
# # Thu thập dữ liệu
# all_weather_data = collect_all_data(start_date, end_date, location, api_key3)

# # In số lượng dữ liệu thu thập được
# print('Number of data:', len(all_weather_data))

# # Lưu dữ liệu vào DataFrame và xuất ra CSV
# data_df = pd.DataFrame(all_weather_data)
# data_df.rename(lambda name: name[0].upper() + name[1:], axis='columns', inplace=True)
# data_df.to_csv('historical_weather_data.csv', index=False)

# # Hiển thị 5 dòng đầu tiên của dữ liệu
# data_df.head()

In [6]:
# Đọc dữ liệu từ file CSV
csv_file_path = 'historical_weather_data.csv'  # Đường dẫn đến file CSV
all_weather_data = pd.read_csv(csv_file_path)

# In số lượng dữ liệu đã tải về
print('Number of data loaded from CSV:', len(all_weather_data))


Number of data loaded from CSV: 732


Datetime           object
DatetimeEpoch       int64
Tempmax           float64
Tempmin           float64
Temp              float64
Feelslikemax      float64
Feelslikemin      float64
Feelslike         float64
Dew               float64
Humidity          float64
Precip            float64
Precipprob        float64
Precipcover       float64
Preciptype         object
Snow              float64
Snowdepth         float64
Windgust          float64
Windspeed         float64
Winddir           float64
Pressure          float64
Cloudcover        float64
Visibility        float64
Solarradiation    float64
Solarenergy       float64
Uvindex           float64
Severerisk        float64
Sunrise            object
SunriseEpoch        int64
Sunset             object
SunsetEpoch         int64
Moonphase         float64
Conditions         object
Description        object
Icon               object
Stations           object
Source             object
dtype: object


In [8]:
# Kiểm tra số lượng giá trị không bị thiếu trong mỗi cột
non_null_counts = all_weather_data.count()

# Tính tỷ lệ phần trăm giá trị thiếu trong mỗi cột
missing_percentage = all_weather_data.isnull().mean() * 100

# Tạo DataFrame từ non-null counts và missing percentage
summary_df = pd.DataFrame({
    'Column Name': all_weather_data.columns,
    'Non-null Count': all_weather_data.count().values,
    'Missing Percentage': all_weather_data.isnull().mean().values * 100
})

# In bảng
print(summary_df)

# Lọc ra các cột có tỷ lệ dữ liệu thiếu lớn hơn 20%
missing_columns = missing_percentage[missing_percentage > 20]
print("\nColumns with more than 20% missing data:")
print(missing_columns)

# Tìm các cột có ít hơn 50 giá trị không bị thiếu
low_data_columns = non_null_counts[non_null_counts < 50]
print("\nColumns with less than 50 non-null values:")
print(low_data_columns)

       Column Name  Non-null Count  Missing Percentage
0         Datetime             732             0.00000
1    DatetimeEpoch             732             0.00000
2          Tempmax             732             0.00000
3          Tempmin             732             0.00000
4             Temp             732             0.00000
5     Feelslikemax             732             0.00000
6     Feelslikemin             732             0.00000
7        Feelslike             732             0.00000
8              Dew             732             0.00000
9         Humidity             732             0.00000
10          Precip             732             0.00000
11      Precipprob             732             0.00000
12     Precipcover             732             0.00000
13      Preciptype             544            25.68306
14            Snow             732             0.00000
15       Snowdepth             732             0.00000
16        Windgust             732             0.00000
17       W

NameError: name 'missing_percentage' is not defined

In [ ]:
# In ra thông tin dữ liệu số cột trước khi loại bỏ
print("Columns before dropping 'Preciptype':")
print(all_weather_data.columns)

# In số lượng cột trước khi loại bỏ
print("Number of columns before dropping 'Preciptype':", len(all_weather_data.columns))

# Loại bỏ cột 'Preciptype' khỏi DataFrame
all_weather_data = all_weather_data.drop(columns=['Preciptype'])

# In ra thông tin dữ liệu sau khi loại bỏ cột
print("Columns after dropping 'Preciptype':")
print(all_weather_data.columns)

# In số lượng cột sau khi loại bỏ
print("Number of columns after dropping 'Preciptype':", len(all_weather_data.columns))

In [ ]:
# Đường dẫn đến file CSV mới
new_csv_file_path = 'cleaned_historical_weather_data.csv'  

# Lưu DataFrame vào file CSV
all_weather_data.to_csv(new_csv_file_path, index=False)

# Xác nhận rằng dữ liệu đã được lưu
print(f"Data has been saved to {new_csv_file_path}.")
